In [1]:
#https://github.com/coinapi/coinapi-sdk/tree/master/python-rest

#https://docs.coinapi.io/#rest-api

In [1]:
import pandas as pd
import numpy as np
from coinapi_v1 import CoinAPIv1
import datetime
from datetime import datetime
import time
from time import mktime as mktime
import calendar

test_key = 'EED0F746-36FB-4CC4-8E7C-527333DFA6FB'

api = CoinAPIv1(test_key)

symbols = api.metadata_list_symbols()

In [2]:
#For every hour, get everything for all platforms and pairs.
#Need a double loop on time and symbols.
#Every call downloads a bunch of high frequency data. 
#It is not clear how to manually set the frequency...
#Simple solution is: just set the limit to 1 (default=100), so no frequency at all.
#Whatever Time is, if it small enough it will be read by coinAPI as some day in 2014.
#For coinAPI, time_start is understood in UTC time (as specified in API)

In [3]:
####Some junk about python time####
#possible_pattern = '%Y%m%dT%H:%M:%S.%fZ'
#pattern='%Y-%m-%dT%H:%M:%S'
#utc_time= utc time in pattern, e.g. '2016-01-01T00:00:00'
#unix_time= unix time since epoch, e.g. '1451606400'
#Commands to switch between one another:
#unix_time = calendar.timegm(time.strptime(utc_time,pattern))
#utc_time=datetime.utcfromtimestamp(unix_time).strftime('%Y-%m-%dT%H:%M:%S')
    

In [24]:
#Inputs: a unix_time; a symbol, from coinAPI symbols.
#Output: a pandas dataframe with historical quotes for a given time and symbol.

def make_dataframe(unix_time, symbol):
    #Convert unix_time in utc with format '%Y-%m-%dT%H:%M:%S'
    utctime = datetime.utcfromtimestamp(unix_time).strftime('%Y-%m-%dT%H:%M:%S')
    quotes_historical_data=api.quotes_historical_data(symbol['symbol_id'],{'time_start': utctime , 'limit':1})
    if quotes_historical_data==[]:
        df=pd.DataFrame()
    else:    
    
        df=pd.DataFrame(data=quotes_historical_data)
        #drop and add new columns.
        df=df.drop(['time_coinapi'], axis=1)
        df=df.drop(['symbol_id'], axis=1)
        df['unix_time']=unix_time
        df['human_time']=utctime
        df['asset_base']=symbol['asset_id_base']
        df['asset_quote']=symbol['asset_id_quote']
        df['exchange_id']=symbol['exchange_id']
        df['type']=symbol['symbol_type']
        Coherence=[]
        if df['ask_price'][0]>df['bid_price'][0]: 
            Coherence.append('')
        else:
            Coherence.append('X')
        df['actual_time_exchange']=df['time_exchange']
        df=df.drop(['time_exchange'],axis=1)
        df['crossed_market']=Coherence
        #move columns around.
        col=df.columns
        col1=col[4:6].append(col[0:4]).append(col[6:])
        df=df[col1]
    
    return(df)

In [20]:
def symbol_merge(unix_time):
    #Given a time, for every symbol request quotes for that symbol at given time.
    #Then merge data in a unique dataframe that contains all the info for that time.
    
    df = pd.DataFrame()
    for symbol in symbols:
        df_temp=make_dataframe(unix_time, symbol)
        df = pd.concat([df, df_temp])
        
    return(df)
        

In [ ]:
def time_merge(unix_time):
    #Given a time, for every minute in the next day call symbol_merge.
    #Merge data in a unique dataframe. There are 86400 seconds in a day, 1440 minutes.
    df=pd.DataFrame()
    for t in range(1440):
        df_temp=symbol_merge(unix_time + 60*t)
        df=pd.concat([df,df_temp])
        
    return(df)    

In [ ]:
def make_daily_dataframe(unix_time):
    df=time_merge(unix_time)
    df.to_csv('quotes_'+str(unix_time)+'.csv', index=False)
    

In [ ]:
#Only left to loop over days.